In [1]:
import csv
from random import shuffle
import os
import progressbar
import time
import datetime

total_patients = 21    # Numero di pazienti totali
folds_offset = 2    # Poiché pesano tanto, conviene parametrizzare anche il "lower bound" del ciclo for
num_folds = 2    # Numero di folds da creare
perc_test = 20    # Percentuale di soggetti da includere nel TESTFOLD_UNLEARNED per ogni fold
patients_test = int( ( total_patients / 100 ) * perc_test )     # Numero di soggetti da inserire nel TESTFOLD_UNLEARNED 
train_split_perc = 0.9    # Percentuale da utilizzare per creare il TRAINFOLD da 'traincount'             

out_sub_folder = "folds_inter"

random_assignment = True    # Se "True", sceglie in maniera randomica i 
                            # soggetti da mettere in train e quelli da mettere in test

prefix = "Biphase_Subject"
suffix = "_norm_windows_20"

subjects_path = "../subjects/min-max/windows_20/tr-False/"
log = subjects_path + out_sub_folder + "/folds_info.txt"
if not os.path.exists(subjects_path + out_sub_folder):
    os.mkdir(subjects_path + out_sub_folder)

with open(log, 'w') as f_log:
    start_time = time.time()
    # Divide il set totale in tre sottogruppi:
        # Train
        # Test Learned
        # Test Unlearned
    # Gli ultimi due sono quelli effettivamente da confrontare poi
    for n in range(folds_offset,num_folds+1):
        train_file = subjects_path + out_sub_folder + "/TrainFold_" + str(n) + ".csv"
        test_file_learned = subjects_path + out_sub_folder + "/TestFold_L_" + str(n) + ".csv"
        test_file_unlearned = subjects_path + out_sub_folder + "/TestFold_U_" + str(n) + ".csv"
        
        # Se vogliamo prendere i soggetti in ordine casuale:
        if random_assignment:
            l = [i for i in range(1,total_patients+1)]    # Riempie il vettore 'l' con numeri da 1 a 'total_patients'
            shuffle(l)    # Li riordina a caso
            l_test = l[:patients_test]    # Il TEST_FOLD è composto dai pazienti da 1 -> 'patient_test'
            l_train = l[patients_test:]    # Il TRAIN_FOLD è composto dai pazienti da 'patients_test' -> fino alla fine
        
        # Se li vogliamo prendere sequenzialmente
        else:
            l_test = []
            for i in range((n-1)*patients_test +1,(n-1)*patients_test + patients_test+1):
                if (i<=total_patients):
                    l_test.append(i)
            l_train = []
            for i in range(1,total_patients+1):
                if not i in l_test:
                    l_train.append(i)
                    
        print("Fold: " + str(n))    
        print("Train: " + str(l_train))
        print("Test: " + str(l_test))
        
        
        with open(train_file, mode='w') as t_out:
            csv_writer = csv.writer(t_out, delimiter=',')
            with open(test_file_learned, mode='w') as t_out_test:
                csv_writer_test = csv.writer(t_out_test, delimiter=',')
                f_log.write("Subjects in Train Fold " + str(n) + ": " + str(l[5:]) + "\n") 
                for i in l_train:    # Apri il soggetto 'i' tra quelli selezionati per il TRAINFOLD
                    with open(subjects_path + prefix + str(i) + suffix +'.csv', mode='r') as s_in:
                        csv_reader = csv.reader(s_in, delimiter=',')
                        row_count = sum(1 for row in csv_reader)
                    train_count = int(row_count*train_split_perc)    # Indica la percentuale da utilizzare come TRAINFOLD - Default è 90%
                    print("Total Rows: " + str(row_count))
                    print("Train rows: " + str(train_count))
                    count = 0
                    with open(subjects_path + prefix + str(i) + suffix +'.csv', mode='r') as s_in:    # Apri il file originale del soggetto 'i' (quello "finestrato")
                        csv_reader = csv.reader(s_in, delimiter=',')
                        for line in progressbar.progressbar(csv_reader):
                            #time.sleep(0.0001)
                            count += 1    # Conta le righe ("finestre") nel file originale
                            if (count<train_count):    # Finché non arriva alla percentuale desiderata
                                csv_writer.writerow(line)    # Scrivi sul TRAINFOLD
                            else:
                                csv_writer_test.writerow(line)    # Altrimenti scrivi su TESTFOLD_LEARNED
        with open(test_file_unlearned, mode='w') as t_out:
            csv_writer = csv.writer(t_out, delimiter=',')
            f_log.write("Subjects in Test Fold " + str(n) + ": " + str(l[:5])+ "\n\n") 
            for i in l_test:    # Apri il soggetto 'i' tra quelli selezionati per il TESTFOLD_UNLEARNED
                with open(subjects_path + prefix + str(i) + suffix +'.csv', mode='r') as s_in:    # Apri il file originale del soggetto 'i' (quello "finestrato")
                    csv_reader = csv.reader(s_in, delimiter=',')
                    for line in csv_reader:
                        csv_writer.writerow(line) # Scrivi su TESTFOLD_UNLEARNED il soggetto 'i' 
        end_time = time.time()        
        time_tot = int(end_time - start_time)
        f_log.write("Elapsed time (Fold " + str(n) + "): " + str(datetime.timedelta(seconds=time_tot)))
        print("Fold " + str(n) + " done!")        # verranno appesi tutti in un unico file alla fine!
        
        
        


Fold: 2
Train: [21, 19, 11, 2, 12, 5, 7, 6, 20, 16, 9, 18, 8, 1, 15, 4, 3]
Test: [14, 17, 10, 13]


- | #                                              | 1699 Elapsed Time: 0:00:00

Total Rows: 24006
Train rows: 21605


| |              #                                | 24005 Elapsed Time: 0:00:01
\ |  #                                             | 3311 Elapsed Time: 0:00:00

Total Rows: 24012
Train rows: 21610


...

Total Rows: 24013
Train rows: 21611


| |             #                                 | 24012 Elapsed Time: 0:00:01
\ |  #                                             | 3466 Elapsed Time: 0:00:00

Total Rows: 24006
Train rows: 21605


| |             #                                 | 24005 Elapsed Time: 0:00:01


Fold 2 done!
